In [ ]:
import numpy
import pandas
import matplotlib.pyplot as plt
import matplotlib

import sys

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from datetime import datetime as dt
import time
import math
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
import urllib.request

tf.set_random_seed(20170605)

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)


#スタート時間を保持
starttime = time.time()

# 学習の設定
#l_of_s         = 20
#n_next         = 5
#l_of_s         = 20
#n_next         = 5

in_out_neurons = 1
hidden_neurons = 300

# 学習用データを抽出する関数
def _load_data(data, n_prev=10, n_next=1, flag=True):
    docX, docY = [], []
    for i in range(n_prev, len(data)-n_next):
        #起点の箇所からn_prevだけ戻った日数分をデータとする
        docX.append(data.iloc[i-n_prev:i].as_matrix())
        #起点の翌日からn_next日数分進んだデータのmax or minをyデータとする
        if flag == True:
            docY.append(max(data.iloc[i+1:i+1+n_next].as_matrix()))
        else:
            docY.append(min(data.iloc[i+1:i+1+n_next].as_matrix()))

    alsX = numpy.array(docX)
    alsY = numpy.array(docY)
    return alsX, alsY


In [ ]:
#data = None
#data = pandas.read_csv(file_name)
#data.columns = ['date', 'open', 'high', 'low', 'close']
#data['date'] = pandas.to_datetime(data['date'], format='%Y-%m-%d')

#close_open_diff = numpy.array( [] )
#for i in range(len(data['date'])-1):
#    close_open_diff = numpy.append(close_open_diff, data['close'][i] - data['open'][i+1])
#
#print('ave:' + str(numpy.mean(close_open_diff)))
#print('std:' + str(numpy.std(close_open_diff)))
#print('max:' + str(numpy.max(close_open_diff)))
#print('min:' + str(numpy.min(close_open_diff)))

#ave = numpy.mean(close_open_diff)
#std = numpy.std(close_open_diff)

#ave_count2 = 0
#ave_count1 = 0
#ave_count3 = 0

#for i in range(len(data['date'])-1):
#    if close_open_diff[i] > ave - std and \
#    close_open_diff[i] < ave + std:
#        ave_count1 += 1
#    if close_open_diff[i] > ave - std * 2 and \
#    close_open_diff[i] < ave + std * 2:
#        ave_count2 += 1
#    if close_open_diff[i] > ave - std * 3and \
#    close_open_diff[i] < ave + std * 3:
#        ave_count3 += 1

#print('total:' + str(len(data['date'])-1))
#print('ave_count1: ' + str(ave_count1) + ' total:' + str(ave_count1/(len(data['date'])-1)))
#print('ave_count2: ' + str(ave_count2) + ' total:' + str(ave_count2/(len(data['date'])-1)))
#print('ave_count3: ' + str(ave_count3) + ' total:' + str(ave_count3/(len(data['date'])-1)))

In [ ]:
# l_of_s
# n_next
# check_treshhold
# file_name
# save_file_name
# activation
# loss_func
# optimizer_func
# validation_split_number
def fit(l_of_s, n_next,check_treshhold,file_name,save_file_name,activation,\
        loss_func,optimizer_func,validation_split_number,\
        train_start_count, train_end_count,test_start_count, test_end_count,\
        data,average_value,diff_value,up_down,check_percent):


    #グラフ化
    #plt.plot(data['date'], data['high'])
    #plt.plot(data['date'], data['low'])
    #plt.show()

    #sys.exit()
    
    #
    #stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

    # データ準備
    #ここではhigh/lowのtrain/testの４つを整備
    #配列的にl_of_sとn_next分の前後にとる
    X_high_train, y_high_train = _load_data(data[['high']].iloc[train_start_count-l_of_s:train_end_count+n_next], \
                                            l_of_s, n_next, True)
    X_high_test , y_high_test  = _load_data(data[['high']].iloc[test_start_count-l_of_s:test_end_count+n_next], \
                                            l_of_s, n_next, True)
    X_low_train , y_low_train  = _load_data(data[['low']].iloc[train_start_count-l_of_s:train_end_count+n_next], \
                                            l_of_s, n_next, False)
    X_low_test  , y_low_test   = _load_data(data[['low']].iloc[test_start_count-l_of_s:test_end_count+n_next], \
                                            l_of_s, n_next, False)


    # ニューラルネットの定義
    #LSTMの定義/high/lowの両方を別々に保持したけど
    #modeは共用してfitだけ違えればいい気がしないでもない
    high_model = Sequential()
    high_model.add(LSTM(hidden_neurons, \
              batch_input_shape=(None, l_of_s, in_out_neurons), \
              return_sequences=False))
    high_model.add(Dense(in_out_neurons))
    #high_model.add(BatchNormalization())
    #high_model.add(Activation("linear"))
    high_model.add(Activation(activation))
    high_model.compile(loss=loss_func, optimizer=optimizer_func)
    #high_model.add(Activation("linear"))
    #high_model.compile(loss="binary_crossentropy", optimizer="adam")

    low_model = Sequential()
    low_model.add(LSTM(hidden_neurons, \
              batch_input_shape=(None, l_of_s, in_out_neurons), \
              return_sequences=False))
    low_model.add(Dense(in_out_neurons))
    #low_model.add(BatchNormalization())
    #low_model.add(Activation("linear"))
    low_model.add(Activation(activation))
    low_model.compile(loss=loss_func, optimizer=optimizer_func)
    #low_model.add(Activation("linear"))
    #low_model.compile(loss="binary_crossentropy", optimizer="adam")

    print("High Fit")
    # 学習
    high_history = high_model.fit(X_high_train, y_high_train, batch_size=100, epochs=300, \
                   validation_split=validation_split_number, callbacks=[early_stopping])

    # テスト結果表示
    high_predicted = high_model.predict(X_high_test)

    print("Low Fit")
    # 学習
    low_history = low_model.fit(X_low_train, y_low_train, batch_size=100, epochs=300, \
                  validation_split=validation_split_number, callbacks=[early_stopping])

    # テスト結果表示
    low_predicted = low_model.predict(X_low_test)

    # high
    val_loss = high_history.history['val_loss']
    plt.rc('font',family='serif')
    fig = plt.figure()
    plt.plot(range(len(high_history.history['val_loss'])), val_loss, label='val_loss', color='black')
    plt.xlabel('epochs')
    plt.savefig('png/' + save_file_name + '_high_' + \
                str(l_of_s) + '_' + str(n_next) + \
                '_' + str(check_treshhold) + '_' + file_name + \
                '_' + activation + '_' + loss_func + \
                '_' + optimizer_func + '_' + str(validation_split_number) + \
                '.png')
    plt.show()

    # low
    val_loss = low_history.history['val_loss']
    plt.rc('font',family='serif')
    fig = plt.figure()
    plt.plot(range(len(low_history.history['val_loss'])), val_loss, label='val_loss', color='black')
    plt.xlabel('epochs')
    plt.savefig('png/' + save_file_name + '_low__' + \
                str(l_of_s) + '_' + str(n_next) + \
                '_' + str(check_treshhold) + '_' + file_name + \
                '_' + activation + '_' + loss_func + \
                '_' + optimizer_func + '_' + str(validation_split_number) + \
                '.png')
    plt.show()




    #high/lowの予想最大/最小値のグラフ（小さくてわからない）
    result = pandas.DataFrame(high_predicted)
    result.columns = ['high_predict']
    result['low_predict'] = low_predicted
    result.plot()
    plt.savefig('png/' + save_file_name + '_result_' + \
                str(l_of_s) + '_' + str(n_next) + \
                '_' + str(check_treshhold) + '_' + file_name + \
                '_' + activation + '_' + loss_func + \
                '_' + optimizer_func + '_' + str(validation_split_number) + \
                '.png')
    plt.show()

    #
    pre_check = []
    temp_check = []
    temp_close_open = []
    temp_close_open_up_win = []
    temp_close_open_up_lost = []
    temp_close_open_down_win = []
    temp_close_open_down_lost = []

    #使わないデータも保持しているが
    #予想したHighの最大値とLowの最小値を起点の日の翌日のOpenの
    #と比較し、最初sに決定した差よりも大きい場合は上がり・下がりと
    #判断する
    #ただし、同時超えた場合はどちらが先に上がるか不明なためカウントしていない
    #もしかすると
    for i in range(len(low_predicted)):
        high_temp  = high_predicted[i] * diff_value + average_value
        low_temp   = low_predicted[i] * diff_value + average_value
        open_temp  = data.loc[i+test_start_count+1, 'open'] * diff_value + average_value
        close_temp = data.loc[i+test_start_count, 'close'] * diff_value + average_value

        if high_temp - close_temp >= check_percent and \
            close_temp - low_temp < check_percent:
            pre_check.append(1)
            temp_check.append(high_temp - open_temp)
            temp_close_open.append(abs(close_temp - open_temp))

        elif close_temp- low_temp >= check_percent and \
            high_temp - close_temp < check_percent:
            pre_check.append(-1)
            temp_check.append(low_temp - open_temp)
            temp_close_open.append(abs(close_temp - open_temp))
        #elif high_temp - open_temp >= check_treshhold and \
        #    open_temp- low_temp >= check_treshhold:

        #    if high_temp - open_temp > open_temp- low_temp:
        #        pre_check.append(1)
        #        temp_check.append(high_temp - open_temp)
        #    elif high_temp - open_temp < open_temp- low_temp:
        #        pre_check.append(-1)
        #        temp_check.append(low_temp - open_temp)
        #    else:
        #        pre_check.append(0)
        #        temp_check.append(0)

        #if high_temp - open_temp >= check_treshhold and \
        #    open_temp - low_temp < check_treshhold and \
        #    close_temp - open_temp <= up_c_o_diff and \
        #    close_temp - open_temp >= down_c_o_diff:
        #    pre_check.append(1)
        #    temp_check.append(high_temp - open_temp)

        #elif open_temp- low_temp >= check_treshhold and \
        #    high_temp - open_temp < check_treshhold and \
        #    close_temp - open_temp <= up_c_o_diff and \
        #    close_temp - open_temp >= down_c_o_diff:
        #    pre_check.append(-1)
        #    temp_check.append(low_temp - open_temp)

        else:
            pre_check.append(0)
            temp_check.append(0)

    up_ok_count = 0
    up_ng_count = 0
    up_ev_count = 0
    down_ok_count = 0
    down_ng_count = 0
    down_ev_count = 0
    high_win = numpy.array([])
    high_lost = numpy.array([])
    low_win  = numpy.array([])
    low_lost  = numpy.array([])

    for i in range(len(pre_check)):
        if pre_check[i] == 1:
            if up_down[i] == pre_check[i]:
                up_ok_count += 1
                high_win = numpy.append(high_win, numpy.array(temp_check[i]))
            elif up_down[i] != pre_check[i] and up_down[i] == -1:
                up_ng_count += 1
                high_lost = numpy.append(high_lost, numpy.array(temp_check[i]))
            else:
                up_ev_count += 1

        elif pre_check[i] == -1:
            if up_down[i] == pre_check[i]:
                down_ok_count += 1
                low_win = numpy.append(low_win, numpy.array(temp_check[i]))
            elif up_down[i] != pre_check[i] and up_down[i] == 1:
                down_ng_count += 1
                low_lost = numpy.append(low_lost, numpy.array(temp_check[i]))
            else:
                down_ev_count += 1


    print('==========')
    print('l_of_s: ' + str(l_of_s) + ' n_next: ' + str(n_next) + \
            ' check_treshhold:' + str(check_treshhold) + ' file_name:' + file_name + \
            ' activation:' + activation + ' loss_func:' + loss_func + \
            ' optimizer_func:' + optimizer_func + ' validation_split_number:' + str(validation_split_number))
    print('UP:')
    print(' WIN  :' + str(up_ok_count))
    print(' LOST :' + str(up_ng_count))
    print(' DRAW :' + str(up_ev_count))
    print('DOWN:')
    print(' WIN  :' + str(down_ok_count))
    print(' LOST :' + str(down_ng_count))
    print(' DRAW :' + str(down_ev_count))
    
    f = open('result/result_' + save_file_name + '.txt', 'a')
    f.write('l_of_s: ' + str(l_of_s) + ' n_next: ' + str(n_next) + \
            ' check_treshhold:' + str(check_treshhold) + ' file_name:' + file_name + \
            ' activation:' + activation + ' loss_func:' + loss_func + \
            ' optimizer_func:' + optimizer_func + ' validation_split_number:' + str(validation_split_number) + \
            '\n')
    f.write('UP: ' + str(up_ok_count) + ' - ' + str(up_ng_count) + ' - ' + str(up_ev_count) + '\n')
    f.write('DN: ' + str(down_ok_count) + ' - ' + str(down_ng_count) + ' - ' + str(down_ev_count) + '\n')
    f.close()
    #print('---------')
    #print('UP ')
    #print('  WIN  :' + str(high_win.mean()) + ' ' + str(high_win.std()))
    #print('  LOST  :' + str(high_lost.mean()) + ' ' + str(high_lost.std()))
    #print('DOWN ')
    #print('  WIN  :' + str(low_win.mean()) + ' ' + str(low_win.std()))
    #print('  LOST  :' + str(low_lost.mean()) + ' ' + str(low_lost.std()))


In [ ]:
def get_data(l_of_s, n_next,check_treshhold,file_name,train_start_count, train_end_count,test_start_count, test_end_count,data):
    # FXデータの読み込み
    #data = None
    #print(file_name)
    #data = pandas.read_csv(file_name)
    #data.columns = ['date', 'open', 'high', 'low', 'close']
    #data['date'] = pandas.to_datetime(data['date'], format='%Y-%m-%d')

    up_down           = []
    up_count          = 0
    down_count        = 0
    even_count        = 0
    #up/downの割合
    loop_flag         = True

    #
    check_add_percent = 0.0020
    check_percent     = 0.02
    if min(data.loc[:, 'low']) > 150:
        check_add_percent = 0.20
        check_percent     = 2.0
    elif min(data.loc[:, 'low']) > 30:
        check_add_percent = 0.020
        check_percent     = 0.2
        #check_percent     = 0.1

    #close_open_diff = numpy.array([])
    #for i in range(train_start_count, test_end_count):
    #    close_open_diff = numpy.append(close_open_diff, numpy.array(data.loc[i-1, 'close'] - data.loc[i, 'open']))
    #print('Close-Open Diff:' + str(close_open_diff.mean()) + ' ' + str(close_open_diff.std()))
    #up_c_o_diff   = close_open_diff.mean() + close_open_diff.std() * 2
    #down_c_o_diff = close_open_diff.mean() - close_open_diff.std() * 2
    #c_o_d_remove_count = 0
    #for i in range(train_start_count, test_end_count):
    #    if data.loc[i-1, 'close'] - data.loc[i, 'open'] >= up_c_o_diff or \
    #        data.loc[i-1, 'close'] - data.loc[i, 'open'] <= down_c_o_diff:
    #        c_o_d_remove_count += 1
    #print("c_o_d_remove_count:" + str(c_o_d_remove_count))

    high_flag = True
    low_flag  = False
    while loop_flag:
        up_count = 0
        down_count = 0
        even_count = 0
        check_percent += check_add_percent
        for i in range(train_start_count, test_end_count):
            #起点の日の翌日のopenの値
            open_value = data.loc[i+1, 'open']
            #起点の日から翌日からn_next日数分のhighの最大値
            max_value = max(data.loc[i+1:i+1+n_next, 'high'])
            #起点の日から翌日からn_next日数分のlowの最小値
            min_value = min(data.loc[i+1:i+1+n_next, 'low'])
            #ここは起点の日の翌日のopenの値と起点の日の翌日から
            #n_next日数分のhighの最高値かlowの最小値が規定の値上に
            #差が広がったカウントを調べる。これで上がった・下がったを
            #回数を調べる

            if max_value - open_value >= check_percent and \
                open_value - min_value < check_percent:
                up_count += 1
            elif open_value - min_value >= check_percent and \
                max_value - open_value < check_percent:
                down_count += 1  
            else:
                even_count += 1

        #(上がった日+下がった日)/全体の日数でcheck_treshholdを超えたか調べる
        #超えていればその値をベースにする
        print('(U+D)/(U+D+E): ' + str(math.floor((up_count + down_count) / \
                                                 (up_count + down_count +even_count) * 100)) + '%')    
        #if high_flag == True:
        #    if (up_count + down_count) / (up_count + down_count +even_count) > check_treshhold:
        #        high_flag = False
        #        low_flag = True
        #elif low_flag == True:
        #    if (up_count + down_count) / (up_count + down_count +even_count) < check_treshhold:
        #        break
        if (up_count + down_count) / (up_count + down_count +even_count) > check_treshhold:
            break

    #上がった・下がったの判定するための数字（もっと言うと円がどれだけ動いたか）
    print('p            : ' + str(check_percent))
    print("UP   COUNT   : " + str(up_count))
    print("DOWN COUNT   : " + str(down_count))
    print("EVNE COUNT   : " + str(even_count))
    print('(U+D)/(U+D+E): ' + str(math.floor((up_count + down_count) / \
                                             (up_count + down_count +even_count) * 100)) + '%')    

    #sys.exit()
    #上がった・下がったの判定用
    #ここではtestの範囲のみ取得
    for i in range(test_start_count, test_end_count):
        open_value = data.loc[i+1, 'open']
        close_value = data.loc[i, 'open']
        #起点の日から翌日からn_next日数分のhighの最大値
        max_value = max(data.loc[i+1:i+1+n_next, 'high'])
        #起点の日から翌日からn_next日数分のlowの最小値
        min_value = min(data.loc[i+1:i+1+n_next, 'low'])
        #ここは起点の日の翌日のopenの値と起点の日の翌日から
        #n_next日数分のhighの最高値かlowの最小値が規定の値上に
        #差が広がったカウントを調べる。これで上がった・下がったを
        #回数を調べる
        if max_value - open_value >= check_percent and \
            open_value - min_value < check_percent:
            up_down.append(1)
        elif open_value - min_value >= check_percent and \
            max_value - open_value < check_percent:
            up_down.append(-1)
        else:
            up_down.append(0)

    #正規化
    max_value = max(data['high'])
    min_value = min(data['low'])
    average_value = (max_value+min_value)/2
    diff_value = max_value - average_value
    print ('max: ' + str(max_value))
    print ('min: ' + str(min_value))
    print ('ave: ' + str(average_value))
    print ('dif: ' + str(diff_value))

    temp_data = data.copy()
    for i in range(len(temp_data['high'].index)):
        temp_data.loc[i, 'high'] = (temp_data.loc[i, 'high'] - average_value) / diff_value
        temp_data.loc[i, 'low']  = (temp_data.loc[i, 'low'] - average_value) / diff_value
        temp_data.loc[i, 'open'] = (temp_data.loc[i, 'open'] - average_value) / diff_value
        temp_data.loc[i, 'close'] = (temp_data.loc[i, 'close'] - average_value) / diff_value

    temp_data = temp_data.sort_values(by='date')
    temp_data = temp_data.reset_index(drop=True)
    temp_data = temp_data.loc[:, ['date', 'open','high', 'low', 'close']]

    return (temp_data,average_value,diff_value, up_down,check_percent)

In [ ]:
def get_date(start_day, end_day, currency_pair):
    #スタート日付
    #終了日を今日に指定
    url       = "https://stooq.com/q/d/l/?s=" + currency_pair + \
                "&d1=" + start_day + "&d2=" + end_day + "&i=d"
    file_name = currency_pair + '_d.csv'
    #取得して、ファイルに保存(よくよく考えると保存しなくてもいいな)
    urllib.request.urlretrieve(url, file_name)
    # FXデータの読み込み
    data = None
    #print(file_name)
    data = pandas.read_csv(file_name)
    data.columns = ['date', 'open', 'high', 'low', 'close']
    data['date'] = pandas.to_datetime(data['date'], format='%Y-%m-%d')

    #train開始終了、test開始終了日を設定
    #データは2001/01/01からとっていますが、さかのぼって調査する関係上
    #20001/01/01からにしています。また買い目対象の調査としてtest_end_dayよりも
    #後の日も参照するためtest_end_dayの取得するデータにはしないでください。
    #trainの終了日とtestの開始日が連続していますが、連続させなくても動作するか
    #検証していません。注意してください
    train_start_day   = dt.strptime('2002-01-01', '%Y-%m-%d')
    train_end_day     = dt.strptime('2013-12-31', '%Y-%m-%d')
    test_start_day    = dt.strptime('2014-01-01', '%Y-%m-%d')
    test_end_day      = dt.strptime('2016-12-31', '%Y-%m-%d')
    train_start_count = -1
    train_end_count   = -1
    test_start_count  = -1
    test_end_count    = -1

    #train/testの開始終了日の配列の場所を調査
    for i in range(len(data['date'])):
        if train_start_count == -1 and data['date'][i] >= train_start_day:
            train_start_count = i
        if train_end_count == -1 and data['date'][i] >= train_end_day:
            train_end_count = i
        if test_start_count == -1 and data['date'][i] >= test_start_day:
            test_start_count = i
        if test_end_count == -1 and data['date'][i] >= test_end_day:
            test_end_count = i
            break

    #前にl_of_s日分、後ろにn_next分日数が必要なので
    #チェック。足りない場合は中止。これを考慮に入れて
    #train/testの開始終了日を設定してください
    #if train_start_count - l_of_s < 0 or \
    #    test_end_count + n_next > len(data['date']):

    #    print("data range over")
    #    sys.exit()

    print('Train Start: ' + str(train_start_count))
    print('Train End  : ' + str(train_end_count))
    print('Test  Start: ' + str(test_start_count))
    print('Test  End  : ' + str(test_end_count))

    return (train_start_count, train_end_count,test_start_count, test_end_count,data)

In [ ]:
import os

l_of_s_list                  = [20,25]
n_next_list                  = [5,7]
check_treshhold_list         = [0.50,0.60]
#activation_list              = ['sigmoid','tanh','linear']
activation_list              = ['tanh']
#loss_func_list               = ['mean_squared_error','mean_absolute_error','mean_squared_logarithmic_error']
loss_func_list               = ['mean_squared_error','mean_absolute_error']
#optimizer_func_list          = ['sgd','adadelta','adam','adamax']
optimizer_func_list          = ['adadelta','adam','adamax']
#validation_split_number_list = [0.1,0.05]
validation_split_number_list = [0.05]

#l_of_s_list                  = [20]
#n_next_list                  = [5]
#check_treshhold_list         = [0.5]
#activation_list              = ['linear']
#loss_func_list               = ['mean_squared_error']
#optimizer_func_list          = ['adam']
#validation_split_number_list = [0.10]

currency_pair_list   = ['usdjpy']

# 結果ファイルの格納
if os.path.exists('result') == False:
    os.mkdir('result')
if os.path.exists('png') == False:
    os.mkdir('png')

save_file_name = 'result/result_' + dt.today().strftime("%Y%m%d%H%M%S") + '.txt'
save_file_name = dt.today().strftime("%Y%m%d%H%M%S")

# fxのデータ取得
start_day     = "20010101"
end_day       =  dt.today().strftime("%Y%m%d")

for currency_pair in currency_pair_list:
    (train_start_count, train_end_count,test_start_count, test_end_count,data) = \
        get_date(start_day, end_day, currency_pair)
    file_name = currency_pair + '_d.csv'

    for l_of_s in l_of_s_list:
        for n_next in n_next_list:
            for check_treshhold in check_treshhold_list:
                #
                (chane_data,average_value,diff_value, up_down,check_percent) = \
                    get_data(l_of_s, n_next,check_treshhold, file_name,train_start_count,\
                             train_end_count,test_start_count, test_end_count,data)
                
                #
                for activation in activation_list:
                    for loss_func in loss_func_list:
                        for optimizer_func in optimizer_func_list:
                            for validation_split_number in validation_split_number_list:
                                #print('l_of_s: ' + str(l_of_s) + ' n_next:' + str(n_next) + ' check_treshhold:' + str(check_treshhold) \
                                #  + ' file_name:' + file_name)
                                print('--------------------------')
                                fit_starttime = time.time()
                                fit(l_of_s, n_next,check_treshhold,file_name,save_file_name,activation,loss_func,optimizer_func,\
                                    validation_split_number,train_start_count, train_end_count,test_start_count, test_end_count,\
                                    chane_data,average_value,diff_value,up_down,check_percent)
                                print(str(math.floor(time.time() - fit_starttime)) + "s")
                                print('')


print(str(math.floor(time.time() - starttime)) + "s")